In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [48]:
today = datetime.today()
in_file = Path.cwd() / "data" / "raw" / "customer_master.xlsx"
url = 'https://github.com/cphalpert/census-regions/blob/master/us%20census%20bureau%20regions%20and%20divisions.csv?raw=True'
report_dir = Path.cwd() / "reports"
report_file = report_dir / "Commissions_Analysis_{today:%b-%d-%Y}.xlsx"
output_file = Path.cwd() / "data" / "processed" / "customer_rep_data.xlsx"

In [8]:
df_customers = pd.read_excel(in_file, sheet_name='customers', dtype = {'zip_code': 'str'})
df_sales_rep = pd.read_excel(in_file, sheet_name='sales')
states = pd.read_csv(url, usecols=[1,2])

In [4]:
df_customers.head()

,company_name,channel,zip_code,city,state,account_num,total_sales
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912
1,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564
2,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488
3,Galaxy Building,retail,85275,Mesa,AZ,AS3124,1193560
4,Resource Innovation Future,retail,97013,Canby,OR,DK1362,958040


In [6]:
df_sales_rep

,first_name,last_name,region,tenure
0,Shannon,Muniz,NorthEast,5.6
1,Leonard,Malcolm,West,3.8
2,Mona,Sutton,Midwest,5.4
3,Mickey,Tyner,South,0.7


In [9]:
states.head()

,State Code,Region
0,AK,West
1,AL,South
2,AR,South
3,AZ,West
4,CA,West


In [15]:
states['Region'].value_counts()

SOUTH        17
WEST         13
MIDWEST      12
NORTHEAST     9
Name: Region, dtype: int64

In [16]:
df_sales_rep['region'].value_counts()

WEST         1
MIDWEST      1
SOUTH        1
NORTHEAST    1
Name: region, dtype: int64

In [14]:
states['Region'] = states['Region'].str.upper()
df_sales_rep['region'] = df_sales_rep['region'].str.upper()

In [17]:
df_sales_rep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   first_name  4 non-null      object 
 1   last_name   4 non-null      object 
 2   region      4 non-null      object 
 3   tenure      4 non-null      float64
dtypes: float64(1), object(3)
memory usage: 256.0+ bytes


In [18]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   State Code  51 non-null     object
 1   Region      51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [19]:
df_customers.head()

,company_name,channel,zip_code,city,state,account_num,total_sales
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912
1,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564
2,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488
3,Galaxy Building,retail,85275,Mesa,AZ,AS3124,1193560
4,Resource Innovation Future,retail,97013,Canby,OR,DK1362,958040


In [20]:
states.head()

,State Code,Region
0,AK,WEST
1,AL,SOUTH
2,AR,SOUTH
3,AZ,WEST
4,CA,WEST


In [24]:
customer_region = pd.merge(df_customers, states, left_on='state', right_on='State Code')
customer_region = customer_region.drop(columns=['State Code'])

In [26]:
customer_region.head()

,company_name,channel,zip_code,city,state,account_num,total_sales,Region
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,SOUTH
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,SOUTH
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,SOUTH
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,SOUTH
4,East Design Hill,retail,66546,Wakarusa,KS,OL0453,1158564,MIDWEST


In [28]:
customer_region.shape

(50, 8)

In [29]:
customer_rep = pd.merge(customer_region, df_sales_rep, left_on='Region', right_on='region')
customer_rep = customer_rep.drop(columns=['region'])

In [30]:
customer_rep.head()

,company_name,channel,zip_code,city,state,account_num,total_sales,Region,first_name,last_name,tenure
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,SOUTH,Mickey,Tyner,0.7
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,SOUTH,Mickey,Tyner,0.7
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,SOUTH,Mickey,Tyner,0.7
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,SOUTH,Mickey,Tyner,0.7
4,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,SOUTH,Mickey,Tyner,0.7


In [31]:
customer_rep.shape

(50, 11)

In [34]:
commission_rate = 0.0079
customer_rep['commission'] = commission_rate * customer_rep['total_sales']
customer_rep['commission'] = customer_rep['commission'].round(2)

In [35]:
customer_rep.head()

,company_name,channel,zip_code,city,state,account_num,total_sales,Region,first_name,last_name,tenure,commission
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,SOUTH,Mickey,Tyner,0.7,9937.50
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,SOUTH,Mickey,Tyner,0.7,8176.90
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,SOUTH,Mickey,Tyner,0.7,12177.74
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,SOUTH,Mickey,Tyner,0.7,10265.66
4,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,SOUTH,Mickey,Tyner,0.7,13141.56


In [36]:
customer_rep['commission'].sum()

999299.96

In [37]:
customer_rep.groupby(['Region']).agg({'commission': 'sum'})

,commission
Region,
MIDWEST,441927.54
NORTHEAST,101517.88
SOUTH,353867.35
WEST,101987.19


In [41]:
customer_rep.groupby(['Region', 'channel']).agg({'commission': ['sum','mean']}).round(0)

commission         
                          sum     mean
Region    channel                     
MIDWEST   reseller   356020.0  59337.0
          retail      85907.0   9545.0
NORTHEAST reseller    49158.0  49158.0
          retail      52360.0  10472.0
SOUTH     partner     55321.0  27660.0
          reseller   107175.0  53588.0
          retail     191371.0   9569.0
WEST      reseller    62042.0  62042.0
          retail      39946.0   9986.0

In [42]:
customer_rep.groupby(['channel']).agg({'commission': ['sum','mean']}).round(0)

commission         
                sum     mean
channel                     
partner     55321.0  27660.0
reseller   574395.0  57440.0
retail     369584.0   9726.0

In [43]:
customer_rep.groupby(['channel']).agg({'commission': ['sum','mean'],
                                      'company_name': ['count']}).round(0)

commission          company_name
                sum     mean        count
channel                                  
partner     55321.0  27660.0            2
reseller   574395.0  57440.0           10
retail     369584.0   9726.0           38

In [46]:
pd.pivot_table(data=customer_rep,
              index=['Region'],
              columns=['channel'],
              aggfunc=['sum'],
              values=['commission'],
              fill_value=0,
              margins=True)

sum                                 
          commission                                 
channel      partner   reseller     retail        All
Region                                               
MIDWEST         0.00  356020.08   85907.46  441927.54
NORTHEAST       0.00   49158.26   52359.62  101517.88
SOUTH       55320.64  107175.38  191371.33  353867.35
WEST            0.00   62041.67   39945.52  101987.19
All         55320.64  574395.39  369583.93  999299.96

In [47]:
customer_rep.head()

,company_name,channel,zip_code,city,state,account_num,total_sales,Region,first_name,last_name,tenure,commission
0,Universal Technology Vision,retail,22910,Charlottesville,VA,AH5590,1257912,SOUTH,Mickey,Tyner,0.7,9937.50
1,Contract Electronics Industries,retail,24153,Salem,VA,GG0303,1035050,SOUTH,Mickey,Tyner,0.7,8176.90
2,Star Interactive,retail,22153,Springfield,VA,UM2244,1541486,SOUTH,Mickey,Tyner,0.7,12177.74
3,Vision People Solutions,retail,24557,Gretna,VA,WL5283,1299450,SOUTH,Mickey,Tyner,0.7,10265.66
4,Studio Pacific Galaxy,retail,79698,Abilene,TX,YR6861,1663488,SOUTH,Mickey,Tyner,0.7,13141.56


In [49]:
customer_rep.to_excel(output_file, index=False)